In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

In [154]:
#Проверяем, что дадасет корректно загрузился
df.head()

,X,Y,month,day,FFMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [155]:
# Укажем тип данных для категориальных параметров
df['X'] = df.X.astype('category')
df['Y'] = df.Y.astype('category')
df['month'] = df.month.astype('category')
df['day'] = df.day.astype('category')

In [156]:
#Загружаем набор данных
df = pd.read_csv("data/forestfires.csv", sep=",", low_memory=False)

In [157]:
#Смотрим, что типы правильно преобразовались
df.dtypes

X          int64
Y          int64
month     object
day       object
FFMC     float64
DMC      float64
DC       float64
ISI      float64
temp     float64
RH         int64
wind     float64
rain     float64
area     float64
dtype: object

In [158]:
# Проверяем, есть ли пропуски
df.isnull().sum()

X        0
Y        0
month    0
day      0
FFMC     0
DMC      0
DC       0
ISI      0
temp     0
RH       0
wind     0
rain     0
area     0
dtype: int64

In [159]:
# Удалим колениарный признак
df = df.drop(['DMC'], axis=1)

In [160]:
# Закодируем категориальные переменные
res_df = pd.get_dummies(df)
res_df

,X,Y,FFMC,DC,ISI,temp,RH,wind,rain,area,...,month_nov,month_oct,month_sep,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,7,5,86.2,94.3,5.1,8.2,51,6.7,0.0,0.00,...,0,0,0,1,0,0,0,0,0,0
1,7,4,90.6,669.1,6.7,18.0,33,0.9,0.0,0.00,...,0,1,0,0,0,0,0,0,1,0
2,7,4,90.6,686.9,6.7,14.6,33,1.3,0.0,0.00,...,0,1,0,0,0,1,0,0,0,0
3,8,6,91.7,77.5,9.0,8.3,97,4.0,0.2,0.00,...,0,0,0,1,0,0,0,0,0,0
4,8,6,89.3,102.2,9.6,11.4,99,1.8,0.0,0.00,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,81.6,665.6,1.9,27.8,32,2.7,0.0,6.44,...,0,0,0,0,0,0,1,0,0,0
513,2,4,81.6,665.6,1.9,21.9,71,5.8,0.0,54.29,...,0,0,0,0,0,0,1,0,0,0
514,7,4,81.6,665.6,1.9,21.2,70,6.7,0.0,11.16,...,0,0,0,0,0,0,1,0,0,0
515,1,4,94.4,614.7,11.3,25.6,42,4.0,0.0,0.00,...,0,0,0,0,0,1,0,0,0,0


In [171]:
# Определим зависимые и независимые переменные
x = pd.concat([res_df.loc[:, 'FFMC' : 'rain'], 
              res_df.loc[:, 'X_1' : 'day_wed']], axis=1)
y = np.log(res_df['area'] + 1)
y=y.astype('int')

KeyError: 'X_1'

In [172]:
# Разделим набор данных на тренировочную и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)

In [ ]:
# Выберем столбцы с числовыми значениями
train_norm = x_train[x_train.columns[0:8]]
test_norm = x_test[x_test.columns[0:8]]

In [ ]:
# Стандартизируем тренировочный набор
std_scale = preprocessing.StandardScaler().fit(train_norm)
x_train_norm = std_scale.transform(train_norm)

## Нейросетевой классификатор

In [ ]:
# Создадим экземпляр класса многослойного персептрона:
# verbose - параметр для выведения лога (loss - функция потерь, которую нам надо минимизировать)
# max_iter - количество эпох обучения
# batch_size - размер батча, маленькой выборкми)
# hidden_layer_sizes - количество нейронов в скрытом слое - первая цифра - нейроны, вторая - слои
# solver -  аргумент для установки алгоритма оптимизации (adam, lbfgs, sgd)
# learning_rate_init - скорость обучения модели (как скоро она сойдется к глобальному или локальному минимумум)
mlpc = MLPClassifier(
    verbose=True, 
    max_iter=25, 
    batch_size=18, 
    hidden_layer_sizes = (1000,3), 
    solver='lbfgs', 
    learning_rate_init = 0.025,
    momentum=0.9)
mlpc.fit(x_train, y_train)
y_pred_mlpc = mlpc.predict(x_test)

In [ ]:
print ('Accuracy MLPClassifier: {}'.format(mlpc.score(x_test, y_test)))

In [ ]:
confmat_model_mlpc = confusion_matrix(y_true=y_test, y_pred=y_pred_mlpc)
print(confmat_model_mlpc)

print("MLPClassifier")
print (classification_report(y_test, y_pred_mlpc))

## Нейросетевой регрессор

In [1]:
mlpr = MLPRegressor(
    verbose=True, 
    max_iter=65, 
    batch_size=18, 
    hidden_layer_sizes = (700,3), 
    solver='lbfgs', 
    learning_rate_init = 0.025,
    momentum=0.9)
mlpr.fit(x_train, y_train)
y_pred_mlpr = mlpr.predict(x_test)

NameError: name 'MLPRegressor' is not defined

In [165]:
print ('Accuracy MLPRegressor: {}'.format(mlpc.score(x_test, y_test)))

Accuracy MLPRegressor: 0.5897435897435898


In [166]:
np.sqrt(mean_squared_error(y_test, y_pred_mlpr))

1.173248176058099

In [167]:
mean_absolute_error(y_test, y_pred_mlpr)

0.9845337018668883

In [168]:
r2_score(y_test, y_pred_mlpr)

-0.0019075392232479338

In [170]:
print('MLPRegressor:')
print('R2_score: ',round(r2_score(y_test, y_pred_mlpr),2))
print('MSE: ',round(mean_squared_error(y_test, y_pred_mlpr),2))
print('MAE: ',round(mean_absolute_error(y_test, y_pred_mlpr),2))

MLPRegressor:
R2_score:  -0.0
MSE:  1.38
MAE:  0.98


## Отчет
1)
Перебор значений параметров не дал впечатляющих результатов. Через увеличение количества нейронов и эпох обучения удалось минимизировть функцию потерь. Однако метрики показывают несопоставимо худшее качество по сравнению с KNeighbors классификатором. Самый лучший результат (precision 0.59, recall 1.00, f1-score 0.74) дал алгоритм оптимизации lbfgs.

2)
Нейросетевой регрессор не справился. При любых значениях параметров получаем отрицательное значение коэффициента детерминации, в то время, как значение должно раскладываться в диапазоне от 0 до 1. Качество модели сопоставимо с качеством Линейного регрессора, но гораздо хуже полиномиальный регрессора.